In [28]:
from PIL import Image
import io
import numpy as np
from package.utils.logger import logger

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.mixture import GaussianMixture
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.kernel_ridge import KernelRidge

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, precision_score, adjusted_rand_score

import pandas as pd
from IPython.display import display

In [2]:

# Bootstrap
raw_dataset = np.load('.ds.tiny/dataset.zip')

dataset = {
    'train': {
        'data': [],
        'names': [],
        'labels': [],
        'unique_labels': [],
    },
    'valid': {
        'data': [],
        'names': [],
        'labels': [],
        'unique_labels': [],
    }
}

images_resize_shape = (128,128)

# For each image we have the path from which we extract the name and the label of the image
for dsKey in raw_dataset.keys():
    splittedKey = dsKey.split('/')

    img_type = splittedKey[2]
    img_label = splittedKey[3]
    img_name = splittedKey[4]
    
    img = Image.open(io.BytesIO(raw_dataset[dsKey])).resize(images_resize_shape, Image.Resampling.LANCZOS)
    img_array = np.asarray(img).reshape(images_resize_shape[0]*images_resize_shape[1], 3)
    
    dataset[img_type]['data'].append(img_array)
    dataset[img_type]['names'].append(img_name)
    dataset[img_type]['labels'].append(img_label)

for img_type in dataset.keys():
    dataset[img_type]['data'] = np.asarray(dataset[img_type]['data'])
    dataset[img_type]['names'] = np.asarray(dataset[img_type]['names'])

    dataset[img_type]['unique_labels'], dataset[img_type]['labels'] = np.unique(np.asarray(dataset[img_type]['labels']), return_inverse=True)

    logger.info([f'data shape({img_type})', dataset[img_type]['data'].shape])
    logger.info([f'data labels({img_type})', dataset[img_type]['labels'].shape])
    logger.info([f'data unique labels({img_type})', dataset[img_type]['unique_labels']])


DEFAULT_LOGGER: 2024-06-06 18:00:41,822 | INFO | 2263878544.py:42 ['data shape(train)', (1500, 16384, 3)]
DEFAULT_LOGGER: 2024-06-06 18:00:41,822 | INFO | 2263878544.py:43 ['data labels(train)', (1500,)]
DEFAULT_LOGGER: 2024-06-06 18:00:41,823 | INFO | 2263878544.py:44 ['data unique labels(train)', array(['apple_pie', 'bibimbap', 'cannoli', 'edamame', 'falafel',
       'french_toast', 'ice_cream', 'ramen', 'sushi', 'tiramisu'],
      dtype='<U12')]
DEFAULT_LOGGER: 2024-06-06 18:00:41,827 | INFO | 2263878544.py:42 ['data shape(valid)', (500, 16384, 3)]
DEFAULT_LOGGER: 2024-06-06 18:00:41,828 | INFO | 2263878544.py:43 ['data labels(valid)', (500,)]
DEFAULT_LOGGER: 2024-06-06 18:00:41,828 | INFO | 2263878544.py:44 ['data unique labels(valid)', array(['apple_pie', 'bibimbap', 'cannoli', 'edamame', 'falafel',
       'french_toast', 'ice_cream', 'ramen', 'sushi', 'tiramisu'],
      dtype='<U12')]


In [22]:
# Dimensionality reduction

n_components_to_test = {
    'PCA': [3, 10, 50, 100, 200, 500, 1200],
    'LDA': [3, 5, 7, 9]    
}

PCAs_instances = {
    'original': {},
    'grayscale': {}
}

LDAs_instances = {
    'original': {},
    'grayscale': {}
}

PCAs_results = {
    'train': {
        'original': {},
        'grayscale': {}
    },
    'valid': {
        'original': {},
        'grayscale': {}
    },
}

LDAs_results = {
    'train': {
        'original': {},
        'grayscale': {}
    },
    'valid': {
        'original': {},
        'grayscale': {}
    },
}

# Grayscaled data
grayscale_train_images = np.mean(dataset['train']['data'], axis=2)
grayscale_valid_images = np.mean(dataset['valid']['data'], axis=2)

for n_components in n_components_to_test['PCA']:
    PCAs_instances['original'][n_components] = []
    PCAs_instances['grayscale'][n_components] = []

    PCAs_results['train']['original'][n_components] = []
    PCAs_results['train']['grayscale'][n_components] = []
    PCAs_results['valid']['original'][n_components] = []
    PCAs_results['valid']['grayscale'][n_components] = []

    PCA_original = [
        make_pipeline(
            StandardScaler(),
            PCA(n_components=n_components)
        ),
        make_pipeline(
            StandardScaler(),
            PCA(n_components=n_components)
        ),
        make_pipeline(
            StandardScaler(),
            PCA(n_components=n_components)
        )
    ]
    
    PCA_grayscale = make_pipeline(
        StandardScaler(),
        PCA(n_components=n_components)
    )

    # Multichannel section
    for i in range(3):
        PCA_original[i].fit(dataset['train']['data'][:,:,i]) 
        PCAs_results['train']['original'][n_components].append(PCA_original[i].transform(dataset['train']['data'][:,:,i]))
        PCAs_results['valid']['original'][n_components].append(PCA_original[i].transform(dataset['valid']['data'][:,:,i]))
        logger.info([f'PCA (channel {i}, {n_components} components): explained_variance_ratio sum', np.sum(PCA_original[i][1].explained_variance_ratio_, axis=0)])

    PCAs_instances['original'][n_components].append(PCA_original)

    # Grayscale section
    PCA_grayscale.fit(grayscale_train_images)

    PCAs_results['train']['grayscale'][n_components] = PCA_grayscale.transform(grayscale_train_images)
    PCAs_results['valid']['grayscale'][n_components] = PCA_grayscale.transform(grayscale_valid_images)

    logger.info([f'PCA (grayscale image, {n_components} components): explained_variance_ratio sum', np.sum(PCA_grayscale[1].explained_variance_ratio_, axis=0)])

for n_components in n_components_to_test['LDA']:
    LDAs_instances['original'][n_components] = []
    LDAs_instances['grayscale'][n_components] = []

    LDAs_results['train']['original'][n_components] = []
    LDAs_results['train']['grayscale'][n_components] = []
    LDAs_results['valid']['original'][n_components] = []
    LDAs_results['valid']['grayscale'][n_components] = []

    LDA_original = [
        make_pipeline(
            StandardScaler(),
            LinearDiscriminantAnalysis(n_components=n_components)
        ),
        make_pipeline(
            StandardScaler(),
            LinearDiscriminantAnalysis(n_components=n_components)
        ),
        make_pipeline(
            StandardScaler(),
            LinearDiscriminantAnalysis(n_components=n_components)
        )
    ]
    
    LDA_grayscale = make_pipeline(
        StandardScaler(),
        LinearDiscriminantAnalysis(n_components=n_components)
    )


    # Multichannel section
    for i in range(3):
        LDA_original[i].fit(dataset['train']['data'][:,:,i], dataset['train']['labels']) 
        LDAs_results['train']['original'][n_components].append(LDA_original[i].transform(dataset['train']['data'][:,:,i]))
        LDAs_results['valid']['original'][n_components].append(LDA_original[i].transform(dataset['valid']['data'][:,:,i]))
        logger.info([f'LDA (channel {i}, {n_components} components): explained_variance_ratio sum', np.sum(LDA_original[i][1].explained_variance_ratio_, axis=0)])

    LDAs_instances['original'][n_components].append(LDA_original)

    # Grayscale section
    LDA_grayscale.fit(grayscale_train_images,dataset['train']['labels'])

    
    LDAs_results['train']['grayscale'][n_components] = LDA_grayscale.transform(grayscale_train_images)
    LDAs_results['valid']['grayscale'][n_components] = LDA_grayscale.transform(grayscale_valid_images)

    logger.info([f'LDA (grayscale image, {n_components} components): explained_variance_ratio sum', np.sum(LDA_grayscale[1].explained_variance_ratio_, axis=0)])


DEFAULT_LOGGER: 2024-06-06 18:45:04,452 | INFO | 1485902721.py:78 ['PCA (channel 0, 3 components): explained_variance_ratio sum', 0.3857434211028181]
DEFAULT_LOGGER: 2024-06-06 18:45:06,473 | INFO | 1485902721.py:78 ['PCA (channel 1, 3 components): explained_variance_ratio sum', 0.34405484140892745]
DEFAULT_LOGGER: 2024-06-06 18:45:08,148 | INFO | 1485902721.py:78 ['PCA (channel 2, 3 components): explained_variance_ratio sum', 0.3741552504145572]
DEFAULT_LOGGER: 2024-06-06 18:45:09,873 | INFO | 1485902721.py:88 ['PCA (grayscale image, 3 components): explained_variance_ratio sum', 0.3495827180560457]
DEFAULT_LOGGER: 2024-06-06 18:45:11,916 | INFO | 1485902721.py:78 ['PCA (channel 0, 10 components): explained_variance_ratio sum', 0.5580101343316143]
DEFAULT_LOGGER: 2024-06-06 18:45:13,631 | INFO | 1485902721.py:78 ['PCA (channel 1, 10 components): explained_variance_ratio sum', 0.519641375420073]
DEFAULT_LOGGER: 2024-06-06 18:45:15,726 | INFO | 1485902721.py:78 ['PCA (channel 2, 10 compo

In [23]:
# Classification - KNN

k_to_test = {
    'PCA': [3, 5, 9, 15, 21, 27, 55, 111, 251],
    'LDA': [3, 5, 9, 15, 21, 27, 55, 111, 251]
}

KNN_PCA_grayscale_stats = []
KNN_LDA_grayscale_stats = []

KNN_PCA_original_stats = []
KNN_LDA_original_stats = []

for k_idx, k in enumerate(k_to_test['PCA']):

    KNN_PCA_grayscale_stats.insert(k_idx,[k])

    for n_components_idx, n_components in enumerate(n_components_to_test['PCA']):
        knn = OneVsOneClassifier(KNeighborsClassifier(k))

        knn.fit(PCAs_results['train']['grayscale'][n_components], dataset['train']['labels'])
        preds = knn.predict(PCAs_results['valid']['grayscale'][n_components])

        accuracy = round(accuracy_score(dataset['valid']['labels'], preds), 3)
        precision = round(precision_score(dataset['valid']['labels'], preds, average='macro'),3)
        
        # logger.info([f"KNN on PCA (grayscale images, k = {k}, {n_components} components)", "accuracy", accuracy, "precision", precision])

        KNN_PCA_grayscale_stats[k_idx].insert(n_components_idx + 1,(accuracy, precision))

KNN_PCA_df = pd.DataFrame(KNN_PCA_grayscale_stats, columns=['k\\PCA (grayscake)'] + n_components_to_test['PCA'])
display(KNN_PCA_df)

for k_idx,k in enumerate(k_to_test['LDA']):
    
    KNN_LDA_grayscale_stats.insert(k_idx,[k])
    
    for n_components_idx, n_components in enumerate(n_components_to_test['LDA']):
        knn = OneVsRestClassifier( KNeighborsClassifier(k))
        
        knn.fit(LDAs_results['train']['grayscale'][n_components], dataset['train']['labels'])
        preds = knn.predict(LDAs_results['valid']['grayscale'][n_components])

        accuracy = round(accuracy_score(dataset['valid']['labels'], preds), 3)
        precision = round(precision_score(dataset['valid']['labels'], preds, average='macro'),3)

        # logger.info([f"KNN on LDA (grayscale images, k = {k}, {n_components} components)", "accuracy", accuracy, "precision", precision])

        KNN_LDA_grayscale_stats[k_idx].insert(n_components_idx + 1,(accuracy, precision))


KNN_LDA_df = pd.DataFrame(KNN_LDA_grayscale_stats, columns=['k\\LDA (grayscale)'] + n_components_to_test['LDA'])
display(KNN_LDA_df)


/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/enrico/anaconda3/envs/uni/lib/pyth

,k\PCA (grayscake),3,10,50,100,200,500,1200
0,3,"(0.154, 0.156)","(0.162, 0.166)","(0.162, 0.192)","(0.15, 0.202)","(0.15, 0.227)","(0.132, 0.192)","(0.132, 0.173)"
1,5,"(0.156, 0.154)","(0.192, 0.192)","(0.162, 0.185)","(0.148, 0.279)","(0.134, 0.208)","(0.142, 0.197)","(0.126, 0.174)"
2,9,"(0.162, 0.162)","(0.186, 0.187)","(0.158, 0.19)","(0.144, 0.198)","(0.144, 0.224)","(0.144, 0.194)","(0.148, 0.223)"
3,15,"(0.184, 0.174)","(0.21, 0.221)","(0.182, 0.225)","(0.158, 0.231)","(0.164, 0.261)","(0.152, 0.254)","(0.144, 0.197)"
4,21,"(0.182, 0.175)","(0.214, 0.223)","(0.18, 0.283)","(0.166, 0.256)","(0.148, 0.191)","(0.138, 0.182)","(0.142, 0.252)"
5,27,"(0.204, 0.198)","(0.214, 0.212)","(0.158, 0.249)","(0.154, 0.307)","(0.132, 0.172)","(0.142, 0.207)","(0.148, 0.215)"
6,55,"(0.202, 0.2)","(0.226, 0.231)","(0.178, 0.209)","(0.172, 0.23)","(0.162, 0.208)","(0.158, 0.2)","(0.16, 0.215)"
7,111,"(0.184, 0.156)","(0.234, 0.236)","(0.172, 0.211)","(0.176, 0.227)","(0.162, 0.221)","(0.158, 0.28)","(0.152, 0.275)"
8,251,"(0.18, 0.188)","(0.202, 0.213)","(0.192, 0.118)","(0.186, 0.181)","(0.194, 0.265)","(0.196, 0.162)","(0.19, 0.146)"


,k\LDA (grayscale),3,5,7,9
0,3,"(0.14, 0.16)","(0.128, 0.121)","(0.136, 0.135)","(0.136, 0.125)"
1,5,"(0.134, 0.141)","(0.132, 0.122)","(0.152, 0.153)","(0.142, 0.136)"
2,9,"(0.138, 0.142)","(0.136, 0.126)","(0.156, 0.157)","(0.148, 0.141)"
3,15,"(0.138, 0.138)","(0.146, 0.146)","(0.148, 0.144)","(0.156, 0.152)"
4,21,"(0.144, 0.145)","(0.138, 0.132)","(0.154, 0.155)","(0.148, 0.143)"
5,27,"(0.148, 0.151)","(0.134, 0.128)","(0.154, 0.157)","(0.148, 0.144)"
6,55,"(0.146, 0.144)","(0.142, 0.133)","(0.152, 0.155)","(0.152, 0.147)"
7,111,"(0.142, 0.139)","(0.136, 0.132)","(0.148, 0.148)","(0.15, 0.146)"
8,251,"(0.144, 0.142)","(0.136, 0.131)","(0.138, 0.137)","(0.15, 0.144)"


In [27]:
# Classification - KRR

kernels_to_test = {
    'PCA': ['linear', 'poly', 'rbf'],
    'LDA': ['linear', 'poly', 'rbf']
}

KRR_PCA_grayscale_stats = []
KRR_LDA_grayscale_stats = []

for kernel_idx,kernel in enumerate(kernels_to_test['PCA']):

    KRR_PCA_grayscale_stats.insert(kernel_idx,[kernel])
    
    for n_components_idx, n_components in enumerate(n_components_to_test['PCA']):#n_components_to_test['PCA']):
        svm = OneVsOneClassifier(KernelRidge(kernel=kernel))

        svm.fit(PCAs_results['train']['grayscale'][n_components], dataset['train']['labels'])

        preds = svm.predict(PCAs_results['valid']['grayscale'][n_components])

        accuracy = round(accuracy_score(dataset['valid']['labels'], preds), 3)
        precision = round(precision_score(dataset['valid']['labels'], preds, average='macro'),3)

        KRR_PCA_grayscale_stats[kernel_idx].insert(n_components_idx + 1,(accuracy, precision))

KRR_PCA_df = pd.DataFrame(KRR_PCA_grayscale_stats, columns=['kernel\\PCA (grayscale)'] + n_components_to_test['PCA'])
display(KRR_PCA_df)

for kernel_idx,kernel in enumerate(kernels_to_test['LDA']):

    KRR_LDA_grayscale_stats.insert(kernel_idx,[kernel])
    
    for n_components_idx, n_components in enumerate(n_components_to_test['LDA']):
        svm = OneVsOneClassifier(KernelRidge(kernel=kernel))

        svm.fit(LDAs_results['train']['grayscale'][n_components], dataset['train']['labels'])

        preds = svm.predict(LDAs_results['valid']['grayscale'][n_components])

        accuracy = round(accuracy_score(dataset['valid']['labels'], preds), 3)
        precision = round(precision_score(dataset['valid']['labels'], preds, average='macro'),3)

        KRR_LDA_grayscale_stats[kernel_idx].insert(n_components_idx + 1,(accuracy, precision))

KRR_LDA_df = pd.DataFrame(KRR_LDA_grayscale_stats, columns=['kernel\\LDA (grayscale)'] + n_components_to_test['LDA'])
display(KRR_LDA_df)

/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/enrico/anaconda3/envs/uni/lib/pyth

,kernel\PCA (grayscale),3,10,50,100,200,500,1200
0,linear,"(0.184, 0.106)","(0.208, 0.22)","(0.19, 0.248)","(0.132, 0.154)","(0.118, 0.098)","(0.144, 0.13)","(0.158, 0.144)"
1,poly,"(0.128, 0.026)","(0.076, 0.078)","(0.13, 0.129)","(0.13, 0.103)","(0.142, 0.146)","(0.142, 0.281)","(0.142, 0.182)"
2,rbf,"(0.138, 0.14)","(0.154, 0.154)","(0.156, 0.146)","(0.146, 0.136)","(0.16, 0.16)","(0.154, 0.154)","(0.136, 0.172)"


/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/enrico/anaconda3/envs/uni/lib/pyth

,kernel\LDA (grayscale),3,5,7,9
0,linear,"(0.114, 0.077)","(0.114, 0.087)","(0.132, 0.085)","(0.134, 0.122)"
1,poly,"(0.092, 0.064)","(0.108, 0.077)","(0.104, 0.101)","(0.112, 0.11)"
2,rbf,"(0.13, 0.104)","(0.134, 0.1)","(0.156, 0.12)","(0.142, 0.126)"


In [ ]:
# Classification - SVM

kernels_to_test = {
    'PCA': ['poly','rbf'],#['linear', 'poly', 'sigmoid'],
    'LDA': ['poly','rbf']#['linear', 'poly', 'sigmoid'],
}

SVM_PCA_grayscale_stats = []
SVM_LDA_grayscale_stats = []

for kernel_idx,kernel in enumerate(kernels_to_test['PCA']):

    SVM_PCA_grayscale_stats.insert(kernel_idx,[kernel])
    
    for n_components_idx, n_components in enumerate(n_components_to_test['PCA']):#n_components_to_test['PCA']):
        svm = OneVsOneClassifier(SVC(kernel=kernel))

        svm.fit(PCAs_results['train']['grayscale'][n_components], dataset['train']['labels'])

        preds = svm.predict(PCAs_results['valid']['grayscale'][n_components])

        accuracy = round(accuracy_score(dataset['valid']['labels'], preds), 3)
        precision = round(precision_score(dataset['valid']['labels'], preds, average='macro'),3)

        SVM_PCA_grayscale_stats[kernel_idx].insert(n_components_idx + 1,(accuracy, precision))

SVM_PCA_df = pd.DataFrame(SVM_PCA_grayscale_stats, columns=['kernel\\PCA (grayscale)'] + n_components_to_test['PCA'])
display(SVM_PCA_df)

for kernel_idx,kernel in enumerate(kernels_to_test['LDA']):

    SVM_LDA_grayscale_stats.insert(kernel_idx,[kernel])
    
    for n_components_idx, n_components in enumerate(n_components_to_test['LDA']):
        svm = OneVsOneClassifier(SVC(kernel=kernel))

        svm.fit(LDAs_results['train']['grayscale'][n_components], dataset['train']['labels'])

        preds = svm.predict(LDAs_results['valid']['grayscale'][n_components])

        accuracy = round(accuracy_score(dataset['valid']['labels'], preds), 3)
        precision = round(precision_score(dataset['valid']['labels'], preds, average='macro'),3)

        SVM_LDA_grayscale_stats[kernel_idx].insert(n_components_idx + 1,(accuracy, precision))

SVM_LDA_df = pd.DataFrame(SVM_LDA_grayscale_stats, columns=['kernel\\LDA (grayscale)'] + n_components_to_test['LDA'])
display(SVM_LDA_df)

In [43]:
# Classification - QDA

kernels_to_test = {
    'PCA': [''],#['linear', 'poly', 'sigmoid'],
    'RAW_DATA': ['']#['linear', 'poly', 'sigmoid'],
}

QDA_PCA_grayscale_stats = []
QDA_original_grayscale_stats = []

for kernel_idx,kernel in enumerate(kernels_to_test['PCA']):

    QDA_PCA_grayscale_stats.insert(kernel_idx,[kernel])
    
    for n_components_idx, n_components in enumerate(n_components_to_test['PCA']):#n_components_to_test['PCA']):
        qda = OneVsOneClassifier(QuadraticDiscriminantAnalysis())

        qda.fit(PCAs_results['train']['grayscale'][n_components], dataset['train']['labels'])

        preds = qda.predict(PCAs_results['valid']['grayscale'][n_components])

        accuracy = round(accuracy_score(dataset['valid']['labels'], preds), 3)
        precision = round(precision_score(dataset['valid']['labels'], preds, average='macro'),3)

        QDA_PCA_grayscale_stats[kernel_idx].insert(n_components_idx + 1,(accuracy, precision))

QDA_PCA_df = pd.DataFrame(QDA_PCA_grayscale_stats, columns=['kernel\\PCA (grayscale)'] + n_components_to_test['PCA'])
display(QDA_PCA_df)

for kernel_idx,kernel in enumerate(kernels_to_test['RAW_DATA']):

    QDA_original_grayscale_stats.insert(kernel_idx,[kernel])

    qda = OneVsOneClassifier(QuadraticDiscriminantAnalysis())

    qda.fit(grayscale_train_images, dataset['train']['labels'])

    preds = qda.predict(grayscale_valid_images)

    print(preds.shape, dataset['valid']['labels'].shape)
    
    accuracy = round(accuracy_score(dataset['valid']['labels'], preds), 3)
    precision = round(precision_score(dataset['valid']['labels'], preds, average='macro'),3)

    QDA_original_grayscale_stats[kernel_idx].insert(n_components_idx + 1,(accuracy, precision))

QDA_original_grayscale_df = pd.DataFrame(QDA_original_grayscale_stats, columns=['','kernel\\Raw images (grayscale)'])
display(QDA_original_grayscale_df)

/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWa

,kernel\PCA (grayscale),3,10,50,100,200,500,1200
0,,"(0.198, 0.187)","(0.244, 0.226)","(0.2, 0.203)","(0.162, 0.188)","(0.106, 0.105)","(0.096, 0.094)","(0.08, 0.083)"


/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/enrico/anaconda3/envs/uni/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWa

(500,) (500,)


,,kernel\Raw images (grayscale)
0,,"(0.088, 0.09)"


In [28]:
# GMM

GMM_n_components_to_test = {
    'PCA': [15, 30, 100, 500]
}

GMM_PCA_score = []

for GMM_n_components_idx,GMM_n_components in enumerate(GMM_n_components_to_test['PCA']):

    GMM_PCA_score.insert(GMM_n_components_idx,[GMM_n_components])
    
    for n_components_idx, n_components in enumerate(n_components_to_test['PCA']):
        gmm = GaussianMixture(n_components=GMM_n_components)

        gmm.fit(PCAs_results['train']['grayscale'][n_components])

        preds = gmm.predict(PCAs_results['valid']['grayscale'][n_components])

        rand_score = round(adjusted_rand_score(dataset['valid']['labels'], preds),3)

        GMM_PCA_score[GMM_n_components_idx].insert(n_components_idx + 1,(rand_score))

GMM_PCA_stats = pd.DataFrame(GMM_PCA_score, columns=['components\\PCA components'] + n_components_to_test['PCA'])

display(GMM_PCA_stats)